In [3]:
import pandas as pd
import csv
from pathlib import Path

In [5]:
DIR = Path("../")

In [52]:
DIALECT = "db_dialect"
csv.register_dialect(
    DIALECT,
    delimiter=",",
    quoting=csv.QUOTE_MINIMAL,
    escapechar="\\",
)

CODE_DIALECT = "code_dialect"

csv.register_dialect(
    CODE_DIALECT,
    delimiter=",",
    quotechar='"',
    quoting=csv.QUOTE_MINIMAL,
)


In [53]:
code = pd.read_csv(DIR / "cleaned-up-code.csv", dialect=CODE_DIALECT)
descriptions = pd.read_csv(DIR / "enhanced-vulnerability-descriptions.csv", dialect=DIALECT)
mitigations = pd.read_csv(DIR / "enhanced-recommendations.csv", dialect=DIALECT)
types = pd.read_csv(DIR / "vulnerability-types.csv", dialect=DIALECT)
types = pd.read_csv(DIR / "functionality.csv", dialect=DIALECT)

In [58]:
print(len(code))
print(len(descriptions))
print(len(mitigations))
print(len(types))

1971
1971
1971
1971


In [59]:
combined_df = pd.concat([
    code,
    descriptions,
    mitigations,
    types,
], axis=1)

In [60]:
combined_df.head(10)

,code,description,recommendation,type
0,"```\n(\n netPnLE36,\n lenderProfitUSDVal...","When a position is liquidated, the liquidator ...","To address the vulnerability, we need to ensur...",token locking
1,```\nuint deltaTime;\n\nif (pos.startLiqTimest...,This vulnerability allows an attacker to manip...,"To mitigate this vulnerability, consider imple...",arithmetic manipulation
2,"```\nfunction _mintInternal(address _receiver,...",This vulnerability allows an attacker to manip...,To prevent the first depositor from being fron...,rounding error
3,```\nuint inputTotalUSDValueE36;\nfor (uint i ...,This vulnerability allows an attacker to manip...,"To mitigate the vulnerability, it is recommend...",arithmetic manipulation
4,```\nfunction increaseLiquidity(IncreaseLiquid...,The UniswapV3NPM contract allows users to incr...,To prevent an attacker from increasing the liq...,arbitrary liquidity manipulation
5,```\nfor (uint i = 0; i < _statuses.length; ) ...,The `SwapHelper.getCalldata()` function retrie...,"To mitigate this vulnerability, implement a co...",data exposure
6,```\nfor (uint i; i < swapParams.length; ) {\n...,The swap calculation when closing a position d...,"When closing a position, it is crucial to accu...",integer overflow
7,"```\nfunction _mintInternal(address _receiver,...",The freeze mechanism in this contract introduc...,"To mitigate the vulnerability, consider implem...",time-based locking
8,```\nstruct tradeInput {\n address spendTok...,The vulnerability allows a malicious operator ...,To prevent a malicious operator from draining ...,reentrancy
9,```\nif (D == 0) {\n uint256 sumDenoms = 0;...,The vulnerability allows a malicious operator ...,To prevent a malicious operator from stealing ...,arithmetic overflow


In [62]:
combined_df.to_csv(DIR / "enhanced-dataset.csv", header=True, index=True)

In [64]:
from datasets import Dataset, load_dataset
import pandas as pd
from common.directories import DATASET_DIR

In [65]:
DATASET_NAME = 'enhanced-vulnerability-audits'

# Add root to path
import sys
current_path = sys.path[0]
root_name = 'data-lifecycle'
root_path = current_path[:sys.path[0].find(root_name) + len(root_name)]
if root_path not in sys.path:
    sys.path.insert(0, root_path)

In [67]:
combined_dataset = Dataset.from_pandas(combined_df)

In [68]:
split_dataset = combined_dataset.train_test_split(test_size=0.15)

In [69]:
split_dataset.save_to_disk(DATASET_DIR / DATASET_NAME)

Saving the dataset (0/1 shards):   0%|          | 0/1675 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/296 [00:00<?, ? examples/s]